<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/Image/10_spectral_transformations.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/Image/10_spectral_transformations.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/Image/10_spectral_transformations.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Spectral transformations
There are several spectral transformation methods in Earth Engine. These include instance methods on images such as `normalizedDifference()`, `unmix()`, `rgbToHsv()` and `hsvToRgb()`. The latter two methods are useful for pan sharpening. For example:

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Add Earth Engine Python script 

In [ ]:
# Load a Landsat 8 top-of-atmosphere reflectance image.
image = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318")
Map.addLayer(
    image,
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.25, "gamma": [1.1, 1.1, 1]},
    "rgb",
)

# Convert the RGB bands to the HSV color space.
hsv = image.select(["B4", "B3", "B2"]).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat(
    [hsv.select("hue"), hsv.select("saturation"), image.select("B8")]
).hsvToRgb()

# Display the pan-sharpened result.
Map.setCenter(-122.44829, 37.76664, 13)
Map.addLayer(
    sharpened, {"min": 0, "max": 0.25, "gamma": [1.3, 1.3, 1.3]}, "pan-sharpened"
)

Map.addLayerControl()
Map

Spectral unmixing is implemented in Earth Engine as the `image.unmix()` method. (For more flexible methods, see the [Array Transformations page](https://developers.google.com/earth-engine/arrays_transformations)). The following is an example of unmixing Landsat 5 with predetermined urban, vegetation and water endmembers:

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7"]
image = ee.Image("LANDSAT/LT05/C01/T1/LT05_044034_20080214").select(bands)
Map.setCenter(-122.1899, 37.5010, 10)  # San Francisco Bay
Map.addLayer(image, {"bands": ["B4", "B3", "B2"], "min": 0, "max": 128}, "image")

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])
Map.addLayer(fractions, {}, "unmixed")
# Map.setCenter(-122.44829, 37.76664, 13)

Map.addLayerControl()
Map